<a href="https://colab.research.google.com/github/lcipolina/foocars/blob/lucia/FooCars_Step_By_Step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is Foocars and why should I care?

Foocars is an inexpensive open source autonomous car driving system. It uses machine learning to create a robust method of driving cars around cones and obstaces.  "Cars" is broad term in this case. It's been used on toy cars, RC Cars, and single seater Power Wheels vehicles. This is an extremely low cost and robust option to get started in understanding autonomous vehicles.

For this project we''ll use the RC car version and work out how the car trains and later makes decisions based on the collected training data.


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from tqdm import tqdm

# https://matplotlib.org/api/image_api.html
# https://matplotlib.org/users/image_tutorial.html
#import matplotlib.image as mpimg
#%matplotlib inline


# The Foocars project 

It's located here: https://foocars.io

The main working repository for the project is here: https://github.com/fubarlabs/foocars

In [0]:
%env USERNAME=lcipolina
!echo $USERNAME
!echo https://github.com/$USERNAME/foocars.git


env: USERNAME=lcipolina
lcipolina
https://github.com/lcipolina/foocars.git


In [0]:
# Create a Fork of the project in GitHub
# Clone your Fork of the project
!git clone https://github.com/$USERNAME/foocars.git

fatal: destination path 'foocars' already exists and is not an empty directory.


In [0]:
!ls

foocars  Pipfile  sample_data


In [0]:
%cd foocars

/content/foocars


In [0]:
!ls


In [0]:
#install pipenv
!pip install pipenv

In [0]:
!pipenv install --system

[pipenv.exceptions.PipenvOptionsError]:       return callback(*args, **kwargs)
[pipenv.exceptions.PipenvOptionsError]:   File "/usr/local/lib/python3.6/dist-packages/pipenv/vendor/click/decorators.py", line 17, in new_func
[pipenv.exceptions.PipenvOptionsError]:       return f(get_current_context(), *args, **kwargs)
[pipenv.exceptions.PipenvOptionsError]:   File "/usr/local/lib/python3.6/dist-packages/pipenv/cli/command.py", line 254, in install
[pipenv.exceptions.PipenvOptionsError]:       editable_packages=state.installstate.editables,
[pipenv.exceptions.PipenvOptionsError]:   File "/usr/local/lib/python3.6/dist-packages/pipenv/core.py", line 1874, in do_install
[pipenv.exceptions.PipenvOptionsError]:       keep_outdated=keep_outdated
[pipenv.exceptions.PipenvOptionsError]:   File "/usr/local/lib/python3.6/dist-packages/pipenv/core.py", line 1230, in do_init
[pipenv.exceptions.PipenvOptionsError]:       "--system is intended to be used for Pipfile installation,
[pipenv.exceptions.Pip

In [0]:
#!pipenv shell

# Environment is setup

We can run the software for training. But first we need to map the data to be acceissble.

 1. Get g drive access to a cars curated training data
 2. Take a look at some of the images
 3. Train on the data set
 4. Select steerstats 
 5. Experiement let's look at steer stats
 6. What happens when we give it an image, what predictin is made
 7. Download the weights and steerstats to the car
 8. Test by running on the car

In [0]:
from google.colab import drive # needs auth code unless made public
drive.mount('/gdrive')

In [0]:
!ls /gdrive/My\ Drive/foocardata/cars/steelroses

In [0]:
!ls /content
!mkdir -p /content/data/cars/steelroses
%cd /content/data/cars/steelroses
!tar xvzf /gdrive/My\ Drive/foocardata/cars/steelroses/steelroses.collected.tar.gz 


In [0]:
!pwd
!ls /content/data/cars/steelroses

In [0]:
%cd /content/foocars/training/


In [0]:
!ls

In [0]:
!python train.py

In [0]:
!python train.py --epochs 10 --save_frequency 2 /content/data/cars/steelroses/collected

## What are steerstats 

Let's look

In [0]:
!ls

## Download weights and steerstats

In [0]:
# get car files and download
!tar cvzf car.weights.tar.gz *.h5 *.npz
!rm *.h5 *.npz

In [0]:
#remove results
!rm *.h5 *.npz

In [0]:
!mkdir -p /content/data/cars/steelroses/model
!tar xvzf car.weights.tar.gz -C /content/data/cars/steelroses/model/

In [0]:
from google.colab import files
files.download('car.weights.tar.gz')

## Examine the weights and steer stats

What does the computer think should happen?

In [0]:
steerstats = np.load('/content/data/cars/steelroses/model/steerstats.npz')


In [0]:
print(type(steerstats))
steerstats.keys()


In [0]:
steerstats['arr_0'][0:10]


## Examine the collected data



In [0]:
# commands.npz
commands = np.load('/content/data/cars/steelroses/collected/commands_2018-12-06_12-54-17.npz'  )
print(type(commands))


In [0]:
print(commands.keys())
commands['arr_0'][0:10]

In [0]:
# IME npz
imu = np.load('/content/data/cars/steelroses/collected/IMU_2018-12-06_12-54-17.npz')
print(type(imu))

In [0]:
print(imu.keys())
imu['arr_0'][0:10]

## What images were collected

In [0]:
images = np.load('/content/data/cars/steelroses/collected/imgs_2018-12-06_12-54-17.npz')
print(type(images))



In [0]:
print(images.keys())
images['arr_0'][0]

In [0]:
#image3 = np.reshape(image3, (8,8,3))
imgplot = plt.imshow(images['arr_0'][0])

In [0]:
## Make a video clip
# Tutorial: https://www.apriorit.com/dev-blog/600-colab-for-video-processing

import cv2
#import skimage.io
#from matplotlib.patches import Polygon
# VIDEO_FORMAT = "MP4V"
# VIDEO_EXT = 'mp4'

VIDEO_FORMAT = 'VP90'
VIDEO_EXT = 'webm'

VIDEO_OUT = '/content/data/output.' + VIDEO_EXT
IMG_IN = '/content/data/cars/steelroses/collected/imgs_2018-12-06_12-54-17.npz'

In [0]:
image = images['arr_0'][0]
print(image.shape)

fourcc = cv2.VideoWriter_fourcc(*VIDEO_FORMAT)
writer = cv2.VideoWriter(VIDEO_OUT, fourcc, 30, (image.shape[1], image.shape[0]), True)


In [0]:
#imgdata = np.load(IMG_IN)['arr_0'].astype('float32')
imgdata = np.load(IMG_IN)['arr_0']
print(f'\nShape: {imgdata.shape}\n')


#ITERATE  the first dimension and get the image from the rest
for index in tqdm(range(0,len(imgdata))):
  img = imgdata[index]
  writer.write(img)

writer.release()
print(f'Done: {VIDEO_OUT}')


In [0]:

!ls /content/data

In [0]:
#Svae video locally
from google.colab import files
files.download(VIDEO_OUT)

In [0]:
from IPython.core.display import Video

Video(VIDEO_OUT)


In [0]:
# The dynamic video tag is not showing. Maybe, do this in javacsript
from IPython.core.display import display, HTML


HTML_SRC = """<video width="400" height="200">
<source src="/content/data/output.webm" type="video/x-webm"></source>
</video>"""
print(HTML_SRC)



             
display(HTML(HTML_SRC))             

# Consulting the Model

In [0]:
import keras
import tensorflow as tf
import concurrent.futures
from dropout_model import model
from defines import *




In [0]:
! ls /content/data/cars/steelroses/model

In [0]:
DATA_DIR = '/content/data/cars'
CAR_NAME = 'steelroses'
WEIGHTS_DIR = DATA_DIR + "/" + CAR_NAME + '/model/'
WEIGHTS_FILE = WEIGHTS_DIR + 'weights_2019-04-06_13-00-23_epoch_4.h5'
STEERSTATS_FILE = DATA_DIR + "/" + CAR_NAME + '/model/steerstats.npz'
print(WEIGHTS_FILE)
print(STEERSTATS_FILE)



In [0]:
# Use Dropout Model
# https://github.com/fubarlabs/foocars/blob/1e6b2bccac5b8595fd053816d9117169d35ae3a1/training/dropout_model.py


import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import Embedding, Input, merge
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.regularizers import l2, l1
from keras.utils.np_utils import to_categorical
from keras.layers.normalization import BatchNormalization
from keras import backend as K



nrows=36
ncols=128
wr=0.00001 # l1 regularizer value
dp=0.125 # dropout rate 

# Note: Dan used the keras functional paradigm to define his network.
# I'm using the sequential paradigm. 
model=Sequential()
frame_in = Input(shape=(3, nrows, ncols), name='img_input')

#we should do a local contrast normalization

print("adding first convolutional layer")
#5x5 convolutional layer with a stride of 2
#model.add(BatchNormalization(input_shape=(nrows, ncols, 3)))
model.add(Conv2D(24, (5, 5), input_shape=(nrows, ncols, 3), strides=(2, 2), activation='elu', padding='same', kernel_initializer='lecun_uniform'))
#model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
model.add(Dropout(dp))

print("adding second convolutional layer")
#5x5 convolutional layer with a stride of 2
#model.add(BatchNormalization())
model.add(Conv2D(32, (5, 5), strides=(2, 2), activation='elu', padding='same', kernel_initializer='lecun_uniform'))
#model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
model.add(Dropout(dp))

print("adding third convolutional layer")
#5x5 convolutional layer with a stride of 2
#model.add(BatchNormalization())
model.add(Conv2D(40, (5, 5), strides=(2, 2), activation='elu', padding='same', kernel_initializer='lecun_uniform'))
#model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
model.add(Dropout(dp))

print("adding fourth convolutional layer")
#3x3 convolutional layer with no stride 
#model.add(BatchNormalization())
model.add(Conv2D(48, (3, 3), strides=(2, 2), activation='elu', padding='same', kernel_initializer='lecun_uniform'))
#model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
model.add(Dropout(dp))

print("adding fifth convolutional layer")
#3x3 convolutional layer with no stride 
#model.add(BatchNormalization())
model.add(Conv2D(48, (3, 3), strides=(2, 2), activation='elu', padding='same', kernel_initializer='lecun_uniform'))
#model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
#model.add(BatchNormalization())
model.add(Dropout(dp))


model.add(Flatten())

print("adding fully connected layer")
#fully connected layer
model.add(Dense(100, activation='elu', kernel_initializer='lecun_uniform'))
model.add(Dropout(dp))

print("adding output layer")
#fully connected layer to output node
model.add(Dense(1, activation='linear', kernel_initializer='lecun_uniform'))

model.compile(loss=['mse'], optimizer=SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True), metrics=['mse'])
print(model.summary())


In [0]:
  model.load_weights(WEIGHTS_FILE)
  model._make_predict_function()
  global g_steerstats
  g_steerstats=np.load(STEERSTATS_FILE)['arr_0']
  
  #g_camera=picamera.PiCamera()
  #g_camera.resolution=(128, 96)
  #g_camera.framerate=FRAME_RATE

In [0]:
#defines image size:
row_offset=30
nrows=36
ncols=128


#imgdata = np.load(IMG_IN)['arr_0'].astype('float32')
imgdata = np.load(IMG_IN)['arr_0']
print(f'\nShape: {imgdata.shape}\n')

index = 1
img = imgdata[index]

# Image must be in the shape the model is using
#ValueError: Error when checking input: expected conv2d_6_input to have shape (36, 128, 3) but got array with shape (96, 128, 3)
# original image shape (200, 96, 128, 3)

#imagerawdata=np.reshape(np.fromstring(s, dtype=np.uint8), (96, 128, 3), 'C')
#imdata=imagerawdata[20:56, :]

immean=img.mean()
imvar=img.std()
g_imageData=np.copy((imgdata-immean)/imvar)
fimg = g_imageData[0]

crop_image=fimg[row_offset:row_offset+nrows, :]
image_mean=crop_image.mean()
image_std=crop_image.std()
guess_image=(crop_image-image_mean)/image_std


#The actual prediction
pred=model.predict(np.expand_dims(guess_image, axis=0))      
steer_command=pred[0][0]*g_steerstats[1]+g_steerstats[0]

print(f'pred: {pred}, steer: {steer_command}')

In [0]:
!ls